In [1]:
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt

In [2]:
in_dir = '1.raw_mri'
binary_dir = '2.train_binary'
out_dir = '3.train_tumor_detected'

In [3]:
wdir = os.listdir(in_dir)
#print(wdir)
total = len(wdir)
#print(total)

In [4]:
Gx= np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
Gy = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

In [8]:
def detect_tumor(img_path, temp_path, output_path):
    img = cv.imread(img_path,1)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    
    orig_img = img
    gray_img = cv.cvtColor( orig_img, cv.COLOR_BGR2GRAY ) 
    median_filtered = cv.medianBlur(gray_img, 5)

    img_sobelx = cv.Sobel(median_filtered,cv.CV_8U,1,0,ksize=3)
    img_sobely = cv.Sobel(median_filtered,cv.CV_8U,0,1,ksize=3)

    img_sobel = img_sobelx + img_sobely + gray_img

    threshold = 137
    maxValue = 255

    th, thresh = cv.threshold(img_sobel, threshold, maxValue, cv.THRESH_BINARY)

    kernel = np.ones((3,3),np.uint8)
    opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 1)

    sure_bg = cv.dilate(opening,kernel,iterations=3)
    dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
    ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)

    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg,sure_fg)

    _, contours, _ = cv.findContours(sure_fg.copy(),cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)

    marker = np.zeros((gray_img.shape[0], gray_img.shape[1]),dtype = np.int32)

    marker = np.int32(sure_fg) + np.int32(sure_bg)

    for id in range(len(contours)):
        cv.drawContours(marker,contours,id,id+2, -1)

    marker = marker + 1

    marker[unknown==255] = 0

    copy_img = orig_img.copy()

    tumor_img = cv.watershed(copy_img, marker)
    #plt.imshow(tumor_img, cmap='gray')
    
    copy_img[marker==-1]=(0,0,255)
    
    plt.imsave(temp_path,tumor_img,cmap='gray')
    
    t_img = cv.imread(temp_path)

    t_img = cv.cvtColor(t_img, cv.COLOR_BGR2RGB)

    tumor_img_gray = cv.cvtColor( t_img, cv.COLOR_BGR2GRAY )
    #plt.imshow(tumor_img_gray, cmap='gray')
    
    ret,tumor_binary_image = cv.threshold(tumor_img_gray,137,1,cv.THRESH_BINARY)
    
    plt.imsave(temp_path,tumor_binary_image,cmap='gray')
    
    orig_img = cv.cvtColor(orig_img, cv.COLOR_RGB2GRAY)
    multi_image = orig_img * tumor_binary_image
    #plt.imshow(multi_image, cmap='gray')
    
    plt.imsave(output_path,multi_image,cmap='gray')


In [9]:
for image_name in os.listdir(in_dir):
    detect_tumor(os.path.join(in_dir,image_name), os.path.join(binary_dir,image_name), os.path.join(out_dir,image_name))

All images are converted to binary and stored in respective folders.